In [15]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from keras.datasets import mnist

In [16]:
# Load MNIST dataset
def load_mnist():
    (X_train, y_train), (X_test, y_test) = mnist.load_data()
    # Normalize pixel values to between 0 and 1
    X_train = X_train.astype('float32') / 255.0
    X_test = X_test.astype('float32') / 255.0
    # Flatten images into 1D vectors
    X_train = X_train.reshape((X_train.shape[0], -1))
    X_test = X_test.reshape((X_test.shape[0], -1))
    # Convert labels to categorical (one-hot encoding)
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)
    return X_train, y_train, X_test, y_test

In [27]:
X_train, y_train, X_test, y_test = load_mnist()
print(X_test.shape)
print(y_train.shape)
print(y_train[1975])

(10000, 784)
(60000, 10)
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]


In [18]:
# Initialize parameters (weights and biases)
def initialize_parameters(input_size, hidden_size, output_size):
    np.random.seed(42)
    W1 = np.random.randn(input_size, hidden_size) * 0.01
    b1 = np.zeros((1, hidden_size))
    W2 = np.random.randn(hidden_size, output_size) * 0.01
    b2 = np.zeros((1, output_size))
    return W1, b1, W2, b2

In [19]:
def relu(Z):
    return np.maximum(0,Z)
    
def relu_derivates(Z): #we need relu derivation for backprop 
    return (Z>0).astype(int)
'''
- returns true if Z is greater than 0, ese returns false
- converts true and false to 1 and 0
'''

#softmax, it might give higher numbers as there is e^z
def softmax(z):
    exp_z = np.exp(z)
    return exp_z/np.sum(exp_z)

In [20]:
# Forward propagation
def forward_propagation(X,parameters): 
    W1, b1, W2, b2 = parameters['W1'], parameters['b1'], parameters['W2'], parameters['b2']
    # Hidden layer
    Z1 = np.dot(W1,X) + b1
    A1 = np.relu(Z1)
    
    # Output layer
    Z2 = np.dot(W2,A1) + b2
    A2 = softmax(Z2)
    
    cache = {"Z1": Z1, "A1": A1, "Z2": Z2, "A2": A2}
    return A2,cache

In [21]:
# Compute cross-entropy loss
def compute_cost(y, y_hat):
    y_hat = y_hat + 1e-15#to avoid log(0)
    loss = - np.sum(y*np.log(y_hat))
    cost = np.mean(loss)
    return cost

In [28]:
def backward_propagation(X,Y,parameters,cache):
    m = X_train.shape[0]
    A1,A2 = cache['A1'],cache['A2']
    W1, W2 = parameters['W1'], parameters['W2']
    
    dZ2= A2 - Y
    dW2 = (1/m)*np.dot(dZ2,A1.T)
    db2 = (1/m)*np.sum(dZ2,axis=1,keepdims=True)
    
    dZ1 = np.dot(W2.T,dZ2)*relu_derivates(A1)
    dW1 = (1/m)*np.dot(dZ1,X.T)
    db1 = (1/m)*np.sum(dZ1,axis=1,keepdims=True)
    
    grads = {"dW1": dW1, "db1": db1, "dW2": dW2, "db2": db2}
    
    return grads